## NSYNC Band name generator

NSYNC is names after the last letter of all the band members' names (sorta). What would other bands be called if names like that.

(Just hit the 'play' button on each code cell to run it.)

*Download word list*

This is a convienient list of token frequencies that I put together at the HathiTrust Research Center, of top-occurring words in 17 million scanned books.



In [ ]:
!wget https://github.com/htrc/HTRC-Useful-Datasets/raw/master/token-frequencies/eng-vocab-1.txt.bz2

In [3]:
import pandas as pd
from IPython.core.display import display, HTML
import re

Load list, and also parse a 'fingerprint' that's easier to compare - alphabetical characters sorted. A no-vowel version is also parsed.

In [4]:
def fingerprint(word, vowels=True):
    chars = [c.lower() for c in list(word) if c.isalpha()]
    if not vowels:
        chars = [c for c in chars if c not in list("aeiou")]
    return "".join(sorted(chars))

words = pd.read_csv('eng-vocab-1.txt.bz2', compression='bz2', names=['word'])
words['length'] = words.word.astype(str).apply(len)
words['fingerprint'] = words.word.astype(str).apply(fingerprint)
words['no_vowel_fingerprint'] = words['fingerprint'].apply(lambda x: fingerprint(x, vowels=False))
words.head(1000).sample(5)

,word,length,fingerprint,no_vowel_fingerprint
938,heat,4,aeht,ht
65,can,3,acn,cn
88,very,4,ervy,rvy
487,mrs.,4,mrs,mrs
981,letters,7,eelrstt,lrstt


In [7]:
#@title Define some functions.

#@markdown Run this cell.

#@markdown `first_name()` - get the first name from a name string 
def first_name(name):
    parts = [x.strip('.') for x in name.split(' ')]
    if len(parts) <= 2:
        return parts[0]
    elif re.match('^[A-Z]\.', name):
        # if has first initial - use second name or second initial
        parts = parts[1]
    else:
        pass
        #print("Trickier name:", name)
    return parts[0]

#@markdown `bandprint(bandlsit)` - take a list of band member names and return a 
#@markdown fingerprint from their last names
def band_print(bandlist, vowels=True):
    names = [first_name(n) for n in bandlist]
    lastchars = "".join([x[-1] for x in names])
    return fingerprint(lastchars, vowels=True)

#@markdown `members_from_string(band_string)` - parse a plain text listing of bandmembers
#@markdown (e.g. `"X, Y, and ZZ"`) into a list
def members_from_string(band_string):
    band_string = band_string.replace(' and ', ' , ')
    return [x.strip() for x in band_string.split(',')]


#@markdown `nsyncer(bandstr,topn=5)` - main function for finding words that match band's fingerprint
def nsyncer(bandstr, topn=5, force_no_vowel=False):
    ''' Return results from string of band members '''
    if ":" in bandstr:
        bandstr = bandstr.split(":")[1].strip()
    band = members_from_string(bandstr)
    matches = words[words.fingerprint == band_print(band)][:topn].word.str.upper()
    if (len(matches) == 0) or force_no_vowel:
        # try no vowel version
        matches = words[words.no_vowel_fingerprint == band_print(band, vowels=False)][:topn]
        condupper = lambda x: x.upper() if x not in list("aeiou") else x
        matches = matches.word.apply(lambda x: "".join([condupper(c) for c in x]))
    return matches

#@markdown `format_vertical(bandstr, bandname)` - Print band member is a way that the 
#@markdown target nsync-style name is lined up and bolded vertically
def format_vertical(bandstr, bandname):
    realname = None
    html = "<p style='font-family:monospace'>"

    if ":" in bandstr:
        realname, bandstr = [x.strip() for x in bandstr.split(":")]
    band = members_from_string(bandstr)
    members = [(name, first_name(name)) for name in band]

    maxfirst = max([len(y) for x,y in members])
    if realname:
        max_width = max([len(x)-len(y)+maxfirst for x,y in members])
        html += realname.upper().center(max_width, '-') + "<br/>"

    remaining = members.copy()
    for letter in bandname.upper():
        for full,first in remaining:
            if first[-1].lower() == letter.lower():
                pad = "&nbsp;" * (maxfirst - len(first))
                html += pad + full.replace(first, f"{first[:-1]}<strong>{first[-1]}</strong>".upper())
                remaining = [(x,y) for x,y in remaining if x!=full]
                html += "<br/>"
                break
        else:
            pad = "&nbsp;" * (maxfirst - 1)
            html += f"<strong><em>{pad}{letter.lower()}</em><br/></strong>"
    display(HTML(html))

# Crunch alt names

In [9]:
bandstr = "Justin Timberlake, JC Chasez, Chris " \
        "Kirkpatrick, Joey Fatone, Lansten Bass"
#display(nsyncer(bandstr))
print("Example of vertical formatting:")
format_vertical(bandstr, "'NSYNC")

Example of vertical formatting:


Example of searching for NSYNC-style band names. You can have the original band name in the input string if it's separated by a colon.

In [10]:
bandstr = "Talking Heads: David Byrne, Jerry Harrison, Chris Frantz, Tina Weymouth" #@param {type: 'string'}
nsyncer(bandstr)

290         DAYS
29099       SAYD
101399    'S-DAY
131757      DYAS
171025      SADY
Name: word, dtype: object

`DAYS` would be fitting for the band that wrote once-in-a-lifetime, but I like the flourish of `S-DAY`. Here's the vertical format:

In [12]:
newname = 'S-DAY' #@param {type: 'string'}
format_vertical(bandstr, newname) 

# Other Bands

In [ ]:
bandstr = "The Who: Roger Daltrey, John Entwistle, Keith Moon, Pete Townshend"
display(nsyncer(bandstr))
format_vertical(bandstr, "HERNIA")

58639     HENR
58741     HERN
89345     REHN
131917    RHEN
182919    HNRE
Name: word, dtype: object

In [ ]:
bandstr = 'Led Zeppelin: Jimmy Page, John Bonham, Robert Plant, John Paul Jones'
display(nsyncer(bandstr))
format_vertical(bandstr, "'NNUITY")

9324     aNNuiTY
10100     NiNeTY
15137     aNToNY
38437     TeNNeY
66762      TYNaN
Name: word, dtype: object

In [ ]:
bandstr = "2012 Beach Boys: Brian Wilson, " \
        "David Marks, Mike Love, Bruce " \
        "Johnston, Al Jardine"
display(nsyncer(bandstr))
format_vertical(bandstr, "ELDEN")

94478     ELDEN
99716     DELEN
106464    LEDEN
128750    EDLEN
137307    ELEND
Name: word, dtype: object

In [ ]:
bandstr = "Grandmaster Flash, Melle Mel, Kidd Creole," \
        "Keef Cowboy, Scorpio and Rahiem"
display(nsyncer(bandstr))
format_vertical(bandstr, "FORMED")

857      FORMED
46815    DEFORM
Name: word, dtype: object

In [ ]:
bandstr = "Velvet Underground and Nico: Lou Reed, " \
        "Sterling Morrison, John Cale, " \
        "Moe Tucker, Nico"
display(nsyncer(bandstr))
format_vertical(bandstr, "UGONE")

184474    GOUNE
188443    UGONE
198405    ONGUE
Name: word, dtype: object

In [ ]:
bandstr = "Jackson 5:Jackie Jackson, Tito Jackson," \
        "Jermaine Jackson, Marlon Jackson," \
        "Michael Jackson"
display(nsyncer(bandstr))
format_vertical(bandstr, "LEONE")

17329     LEONE
109738    ENLOE
180147    OLENE
200826    ENOLE
Name: word, dtype: object

In [ ]:
bandstr = 'The Beatles: John Lennon, Paul McCartney, Ringo Starr, George Harrison'
display(nsyncer(bandstr))
format_vertical(bandstr, "LEON")

8223     LEON
11948    LONE
16109    NOEL
63061    ENOL
63701    ELON
Name: word, dtype: object

In [ ]:
bandstr = 'The Beatles: John Lennon, Paul McCartney, Ringo Starr, George Harrison, George Martin'
display(nsyncer(bandstr))
format_vertical(bandstr, "LEONE")

17329     LEONE
109738    ENLOE
180147    OLENE
200826    ENOLE
Name: word, dtype: object

In [ ]:
bandstr = "Dirty Projectors: David Longstreth," \
        "Felicia Douglass, Maia Friedman," \
        "Mike Daniel Johnson, Kristin Slipp"
display(nsyncer(bandstr))
format_vertical(bandstr, "DANAE")

79547     DANAE
134961    ADENA
167966    DAANE
193886    ANADE
Name: word, dtype: object

In [ ]:
nsyncer('Nirvana: Kurt Cobain, Krist Novoselic, Dave Grohl')

26988    ETT
32051    TET
43399    TTE
Name: word, dtype: object

In [ ]:
bandstr = "Radiohead: Thom Yorke, Jonny Greenwood,"\
        "Colin Greenwood, Ed O'Brien and Philip Selway"
display(nsyncer(bandstr))
format_vertical(bandstr, "eP'NDYMa")

121780    ePeNDYMa
Name: word, dtype: object

[Ependyma](https://en.wikipedia.org/wiki/Ependyma) is fitting, another 'head'-related word.

In [ ]:
bandstr = "Destiny's Child: Beyonce Knowles," \
        "Kelly Rowland, Michelle Williams"
display(nsyncer(bandstr))
format_vertical(bandstr, "EYE")

1298      EYE
36125     YEE
166863    EEY
Name: word, dtype: object

In [ ]:
bandstr = "Justin Timberlake, JC Chasez, Chris " \
        "Kirkpatrick, Joey Fatone, Lance Bass"
display(nsyncer(bandstr))
# this isn't a real word?
format_vertical(bandstr, "SYNCE")

180233    SYNCE
Name: word, dtype: object